# Selecting a model

As mentioned in the first tutorial, a model can be thought as a mathematical implementation of the electrochemical and transport phenomena occuring in a real battery cell.
The implementation consist of a system of partial differential equations and their corresponding parameters, constants, boundary conditions and assumptions.

The default Lithium-Ion Battery Model corresponds to a basic P2D model, where neither current collectors, degradation nor thermal effects are considered.
BattMo has implemented several variants of the Lithium-Ion Battery Model, which can be accessed by *configuring the model object*. In this tutorial, we’ll configure a
P2D model with degradation driven by SEI (Solid Electrolyte Interphase) growth.

### Load BattMo and Model Settings

In [ ]:
using BattMo, GLMakie

Let’s begin by loading the default model settings for a P2D simulation. This will return a ModelSettings object:

In [ ]:
model_settings = load_model_settings(; from_default_set = "P2D")
nothing #hide

We can inspect all current settings with:

In [ ]:
model_settings.all

By default, the "SEIModel" parameter is set to false. Since we want to observe SEI-driven degradation effects, we’ll specify which SEI model we'd like to use, and with that enable the use of
the SEI model during the simulation. Let's have a look at which models are available to include in the settings:

In [ ]:
print_submodels()

For the SEI model, we can see there's one model to enable which is the "Bolay" model. We enable it in the model settings:

In [ ]:
model_settings["SEIModel"] = "Bolay"
model_settings.all

### Initialize the Model
Let’s now create the battery model using the modified settings:

In [ ]:
model = LithiumIonBattery(; model_settings);

When setting up the model, the LithiumIonBattery constructor runs a validation on the model_settings.
In this case, because we set the "SEIModel" parameter to true, the validator provides a warning that we should define which SEI model we would like to use.
If we ignore any warnings and pass the model to the Simulation constructor then we get an error. Let's create such a situation:

In [ ]:
model_settings["SEIModel"] = "Bola"


model = LithiumIonBattery(; model_settings);

We get a warning that a validation issue has been encountered. For now we ignore it:

In [ ]:
cell_parameters_sei = load_cell_parameters(; from_default_set = "Chen2020")
cccv_protocol = load_cycling_protocol(; from_default_set = "CCCV")

try  # hide
	sim = Simulation(model, cell_parameters_sei, cccv_protocol)
catch err # hide
	showerror(stderr, err) # hide
end  # hide

As expected, this results in an error because we didn't specify the SEI model correctly.

### Specify SEI Model and Rebuild
Let's resolve the issue again and run the simulation:

In [ ]:
model_settings["SEIModel"] = "Bolay"
nothing # hide

Now rebuild the model:

In [ ]:
model = LithiumIonBattery(; model_settings);

Now we can setup the simulation and run it.

In [ ]:
sim = Simulation(model, cell_parameters_sei, cccv_protocol)
output = solve(sim)
nothing # hide

## Plot of voltage and current

In [ ]:
plot_dashboard(output; plot_type = "simple")

## Plot of SEI length

We recover the SEI length from the `state` output

In [ ]:
states = get_output_states(output)
seilength = states[:SEIThickness]

We can plot it using the plot_ouput function

In [ ]:
ne_index = sim.settings["GridResolutionNegativeElectrodeCoating"]

plot_output(output,
	["SEIThickness vs Time at Position index $ne_index",
		"Voltage vs Time"],
	layout = (2, 1))

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*